In [1]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset
import torch
import torchaudio
from transformers import WhisperProcessor, pipeline
from datasets import load_dataset
import evaluate
import tempfile
import os



In [2]:
# Step 1: Load the common_voice dataset for Tamil
dataset = load_dataset("mozilla-foundation/common_voice_11_0", "mr", split="test", trust_remote_code=True)


The repository for mozilla-foundation/common_voice_11_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_11_0.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 2245it [00:00, 110367.37it/s]


Generating validation split: 0 examples [00:00, ? examples/s]


Reading metadata...: 1682it [00:00, 94826.66it/s]


...


Reading metadata...: 1816it [00:00, 78261.27it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 2819it [00:00, 108497.60it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 2237it [00:00, 117189.47it/s]


In [4]:
import torchaudio

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["sentence"]
    return batch


In [5]:
dataset = dataset.map(speech_file_to_array_fn, remove_columns=dataset.column_names)

Map:   0%|          | 0/1816 [00:00<?, ? examples/s]

In [3]:
import librosa
import numpy as np

def resample(batch):
    speech_array = np.asarray(batch["speech"])
    batch["speech"] = librosa.resample(speech_array, orig_sr=48_000, target_sr=16_000)
    batch["sampling_rate"] = 16_000
    return batch

In [ ]:
dataset = dataset.map(resample, num_proc=4)

In [ ]:
processor = WhisperProcessor.from_pretrained("openai/whisper-large")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large")
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language="mr", task="transcribe")


In [ ]:
def transcribe(batch):
    input_features = processor(batch["speech"], sampling_rate=batch["sampling_rate"], return_tensors="pt").input_features 

    # generate token ids
    with torch.no_grad():
        predicted_ids = model.generate(input_features)
    # decode token ids to text
    #transcription = processor.batch_decode(predicted_ids, skip_special_tokens=False)
    #print(transcription)
    batch["transcription"] = processor.batch_decode(predicted_ids, skip_special_tokens=True)
    #print(transcription)
    return batch